In [ ]:
from keras.datasets import cifar10

(Xtr, Ytr), (Xte, Yte) = cifar10.load_data() # a magic function we provide
print(Xtr)
# flatten out all images to be one-dimensional
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) # Xtr_rows becomes 50000 x 3072
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3) # Xte_rows becomes 10000 x 3072
print(Xtr_rows)

In [29]:
import math
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randn

In [30]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    e = np.exp(x-np.max(x, axis=1, keepdims=True))
    return e / np.sum(e, axis=1, keepdims=True)
def cross_entropy_error(y,t):
    return -np.sum(t * np.log(y + 1e-9)) / y.shape[0]

In [31]:
class Dense:
  def __init__(self, input_size, output_size, initializer='random'):
      self.W = 0.1 * np.random.randn(input_size, output_size)
      self.b = 0.1 * np.zeros(output_size)
      self.x = None
      self.y = None
      self.dW = None # W gradient
      self.db = None # b gradient

  def forward(self, x):
      self.x = x # x값을 기억해 둠 (gradient 를 구할 때 사용)
      self.y = np.dot(self.x, self.W) + self.b
      return self.y

  def backward(self, d_out, learning_rate):
      self.dW = np.dot(self.x.T, d_out)
      self.db = np.sum(d_out, axis=0)
      d_x = np.dot(d_out, self.W.T)
      self.W -= learning_rate * self.dW
      self.b -= learning_rate * self.db
      return d_x

In [32]:
# last layer
class SoftmaxWithLoss:
  def __init__(self):
      self.error = None
      self.y = None # 예측 답안
      self.t = None # 모범 답안

  def forward(self,x):
      self.y = softmax(x)
      return self.y

  def loss(self, t):
      self.t = t
      self.error = cross_entropy_error(self.y, self.t)
      return self.error

  def backward(self, d_out=1, learning_rate=None): 
      batch_size = self.t.shape[0]
      d_x = (self.y - self.t) / batch_size
      return d_x

In [33]:
class Sigmoid:
  def __init__(self):
      self.y = None

  def forward(self,x):
      self.y = sigmoid(x)
      return self.y

  def backward(self, d_out, learning_rate=None):
      return d_out * (1.0 - self.y) * self.y

In [34]:
class Relu:
  def __init__(self):
      self.mask = None

  def forward(self, x):
      self.mask = (x<=0)
      out = x.copy()
      out[self.mask] = 0
      return out

  def backward(self, d_out, learning_rate=None):
      d_out[self.mask] = 0
      d_x = d_out
      return d_x

In [35]:
# one-hot encoder
def onehot(y, n_class):
    vectors = np.zeros((len(y), n_class))
    for i, label in enumerate(y):
        vectors[i, label] = 1
    return vectors

In [37]:
Xtr_rows = (Xtr_rows - 127.5) / 255
Xte_rows = (Xte_rows - 127.5) / 255

In [38]:
print('Training set')
print('global mean = %.5f' % np.mean(Xtr_rows))
print('global std  = %.5f' % np.std(Xtr_rows))
print('global max = %.5f' % np.max(Xtr_rows))
print('global min = %.5f' % np.min(Xtr_rows))

print()
print('Test set')
print('global mean = %.5f' % np.mean(Xte_rows))
print('global std  = %.5f' % np.std(Xte_rows))
print('global max = %.5f' % np.max(Xte_rows))
print('global min = %.5f' % np.min(Xte_rows))

Training set
global mean = -0.02664
global std  = 0.25157
global max = 0.50000
global min = -0.50000

Test set
global mean = -0.02342
global std  = 0.25122
global max = 0.50000
global min = -0.50000


In [39]:
n_dim = Xtr_rows.shape[1]
n_class = 10

input_layer = Dense(n_dim,100)
output_layer = Dense(100,n_class)
Sigmoid = Sigmoid()
Relu = Relu()

print('shape of W =', input_layer.W.shape)
print('shape of b =', input_layer.b.shape)

print('shape of W =', output_layer.W.shape)
print('shape of b =', output_layer.b.shape)



shape of W = (3072, 100)
shape of b = (100,)
shape of W = (100, 10)
shape of b = (10,)


In [40]:
# sample mini-batch
batch = Xtr_rows[:, :] # the first 4 images from training set
print('shape of mini batch =', batch.shape)

shape of mini batch = (50000, 3072)


2-layer Neural networks 에 cifer-10 데이터 삽입

In [ ]:
# output
output = input_layer.forward(batch)
print('shape of output =', output.shape) # each row assigns 10 numbers 
print(output)


In [ ]:
output = Relu.forward(output)
print(output)

In [ ]:
output = output_layer.forward(output)
print(output)

In [ ]:
answer = Ytr[:, :]
print('shape of mini batch =', answer.shape)
print(answer)

In [ ]:
answer = onehot(Ytr[:], n_class)
print(answer)

In [46]:
loss = SoftmaxWithLoss()
loss.forward(output)
first_loss = loss.loss(answer)
print(first_loss)

2.736595812491496


In [47]:
# model prediction
np.argmax(output, axis=1)

array([1, 1, 5, ..., 7, 0, 5])

backpropatation

In [48]:
learning_rate = 0.1
q = loss.backward()
w = output_layer.backward(q,learning_rate)
e = Relu.backward(w)
r = input_layer.backward(e,learning_rate)

In [ ]:
# output
output2 = input_layer.forward(batch)
print('shape of output =', output2.shape) # each row assigns 10 numbers 
print(output2)

In [ ]:
output2 = Relu.forward(output2)
print(output2)

In [ ]:
output2 = output_layer.forward(output2)
print(output2)

In [52]:
loss = SoftmaxWithLoss()
loss.forward(output2)
second_loss = loss.loss(answer)
print(first_loss)
print(second_loss)

2.736595812491496
2.5585713541854216


In [53]:
# model prediction
np.argmax(output2, axis=1)

array([1, 1, 5, ..., 7, 0, 5])

반복해서 학습



In [54]:
for i in range(100):
  q = loss.backward()
  w = output_layer.backward(q,learning_rate)
  e = Relu.backward(w)
  r = input_layer.backward(e,learning_rate)
  output = input_layer.forward(batch)
  output = Relu.forward(output)
  output = output_layer.forward(output) 
  loss.forward(output)
  while_loss = loss.loss(answer)
  print(while_loss)

2.466107610513327
2.402806162124192
2.355920981228955
2.319034025956047
2.288588111512367
2.262575142625908
2.2398385632353217
2.219615752317207
2.201410344481484
2.1848711977602995


test predict

In [55]:
batch = Xte_rows
print('shape of mini batch =', batch.shape)

shape of mini batch = (10000, 3072)


In [56]:
output = input_layer.forward(batch)
output = Relu.forward(output)
output = output_layer.forward(output)

In [57]:
test_answer = Yte[:,:]
print(test_answer)
test_answer = onehot(test_answer,10)

[[3]
 [8]
 [8]
 ...
 [5]
 [1]
 [7]]


In [58]:
loss_test = SoftmaxWithLoss()
loss_test.forward(output)
first_loss = loss_test.loss(test_answer)
print(first_loss)

2.177953557876951


In [59]:
np.argmax(output, axis=1)

array([5, 1, 8, ..., 6, 5, 7])